In [ ]:
import os
import fitz
import re
import json

In [ ]:
def parse_pdf_to_json_27_06_2011_N_171(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""

    # Пропустить первую страницу
    for page in doc[1:]:
        text += page.get_text()

    # Найти начало полезного текста
    start_match = re.search(r"\bI\.\s+Общие положения\b", text)
    if not start_match:
        raise ValueError("Не найдена начальная глава 'I. Общие положения'")
    text = text[start_match.start():]  # отрезаем бесполезный текст до начала главы

    # Разделение на главы
    chapter_pattern = r"(?:^|\n)([IVXL]+\.)\s+([^\n]+)"  # например, 'I. Общие положения'
    chapter_matches = list(re.finditer(chapter_pattern, text))

    chapters = []
    for i, match in enumerate(chapter_matches):
        chapter_number = match.group(1).strip()
        chapter_title = match.group(2).strip()

        start = match.end()
        end = chapter_matches[i + 1].start() if i + 1 < len(chapter_matches) else len(text)
        chapter_text = text[start:end].strip()

        # Разбить на пункты
        paragraph_pattern = r"\n?(\d+(\.\d+)*\.)\s+(.+?)(?=\n\d|$)"
        paragraphs = []
        for para_match in re.finditer(paragraph_pattern, chapter_text, flags=re.DOTALL):
            number = para_match.group(1).strip()
            content = para_match.group(3).strip().replace('\n', ' ')
            paragraphs.append({
                "номер": number,
                "текст": content
            })

        chapters.append({
            "глава": chapter_number,
            "заголовок": chapter_title,
            "пункты": paragraphs
        })

    result = {
        "документ": pdf_path.split("/")[-1],
        "главы": chapters
    }

    return result

In [ ]:
pdf_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_pdf\\27_06_2011_N_171.pdf"
outp_path = "C:\\Users\\My Computer\\Desktop\\Work\\Learn\\OPK_Project\\documents_json\\27_06_2011_N_171.json"

data = parse_pdf_to_json_27_06_2011_N_171(pdf_path)

with open(outp_path, "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print(f"Документ успешно сохранён в {outp_path}")

Документ успешно сохранён в C:\Users\My Computer\Desktop\Work\Learn\OPK_Project\documents_json\27_06_2011_N_171.json
